In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
from skimage import io

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda


In [ ]:
def map_label(l):
  d={'rotated_right':0, 'rotated_left':1, 'upright':2, 'upside_down':3}
  return d[l]

In [ ]:
class create_dataloader(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.data.fn[idx])
        image = io.imread(img_name)
        label = map_label(self.data.label[idx])
        image = torch.from_numpy(image)
        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
train_dataloader = create_dataloader(csv_file='/content/drive/My Drive/Assignments/Deeper_Systems/data/train.truth.csv',
                                    root_dir='/content/drive/My Drive/Assignments/Deeper_Systems/data/train')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 13 * 13, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 13 * 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = nn.functional.softmax(self.fc3(x))
        return x

net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, batch in enumerate(train_dataloader):
        # get the inputs; data is a list of [inputs, labels]
        image= batch['image']
        label= batch['label']
        label= torch.LongTensor([label])

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        image = torch.reshape(image, (1,image.shape[2],image.shape[0], image.shape[1]))
        logits = net(image.float())
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for batch in testloader:
        image= batch['image']
        label= batch['label']
        image = torch.reshape(image, (1,image.shape[2],image.shape[0], image.shape[1]))
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))